- now we're going to go a level lower, where we actually want to find a specific trajectory that the car can take
- a trajectory is not just a curve that the car can follow but also a time sequence in which we say how fast the car should go
- in finding trajectory, there are many many important things to watch out for
  - the most important one is don't crash--don't collide with something else
  - also important is things like passenger comfort--you don't want to do crazy jerk, a trajectory that passes both back and forth, you want to make this smooth and elegant as possible

- in this lesson, I'm going to teach you about continuous trajectory planning, and more specifically, how to generate draggable trajectories
- first, we will define what the motion planning problem is and discuss a few important concepts and priorities regarding motion planning algorithms
- then we'll do a quick review of A* to get you ready for the first new algorithm we will cover which is called Hybrid A*
  - as name suggests, Hybrid A* isn't purely discrete or continues
- next, we explore sampling based method called polynomial trajectory generation, which is really useful for highway driving

# The Motion Planning Problem

- in the first lesson you saw some of the best planning algorithms which solve the motion planning problem but we never formally defined that problem--I'd like to do that now
- there is a word that you might encounter quite a bit if you start reading material regarding motion planning algorithm--that word is *configuration space*, which defines all the possible configurations of a robot in the given world
- consider the maze world you saw in the first lesson where these worlds were all 2D grids, the robot configuration was sometimes two dimensional when we presented it as an (x, y) point, and sometimes three dimensional when also including the robot's heading
- in fact, the configuration space for vehicle can become even larger depending on what motion planning algorithms we decide to use


- with this idea of configuration space in mind, we can define a motion planning problem as follows:
- we're given:
  - a start configuration $q_{start}$
  - a goal configuration $q_{goal}$
  - some constraints describing how the vehicle is allowed to move, its dynamics and the description of the environment
- here, it is important to understand how this connects to the other decision-making modules that you have recently learned about
- usually, the start configuration is the current configuration of the car given to us by the localization value and the sensors that give us information about car location, speed, acceleration, etc.
- the behavior layer gives us a desired end configuration, and maybe some constraints regarding where to go and at which speed
- the prediction completes this problem by giving us information about how the obstacle region will evolve in time
- this way, the sequence of actions that we generate takes into account other vehicles and pedestrian actions and if we're working with a more sophisticated prediction module, how our actions might influence them


- the motion planning problem can then be defined as: "find a sequence of feasible movements in the configuration space that moves the robot from a start configuration to an end configuration without hitting any obstacles"

# Properties of Motion Planning Algorithms

- when discussing planning algorithms, there are two important properties that we like to talk about
- the first one is called completeness
  - this means that if a solution to the motion planning problem exists, the planner will find it
  - if a solution does not exist, the planner will terminate and report that no solution exists
- the second one is called optimality
  - a planning algorithm is optimal if it always returns the sequence which minimizes some cost function

# Types of Motion Planning Algorithms

- there are many classes of motion planning algorithms and today we'll focus on one of these classes, but it's worth mentioning the others
- combinatorial methods usually consist in dividing the free space into small pieces and first solving the motion planning problem by connecting these atomic elements
  - they are very intuitive ways to find initial approximate solution, but they usually do not scale well for large environments

<img src="resources/combinatorial_methods.png"/>

- next, potential fields are reacting methods
  - each obstacle is going to create a sort of anti-gravity field which makes it harder for the vehicle to come close to it
  - for example, you could imagine using this idea around pedestrians or bikes to encourage your planning algorithm to find trajectories that stay away from them
  - the main problem with most potential field methods is that they sometimes push us into local minima which can prevent us from finding a solution

<img src="resources/potential_field_methods.png"/>

- optimal control consists in trying to solve the motion planning problem and the controlling input generation in one algorithm
  - using a dynamic model of a vehicle or start configuration and end configuration, we want to generate a sequence of inputs, for example, steering angle and throttle inputs that would lead us from start to end configuration while optimizing a cost function relative to the control inputs such as minimizing gas consumption and relative to the configuration of the car, such as staying at a distance from other vehicles
  - there are a lot of very nice ways to do that--most of them based on numerical optimization methods
  - however, it is hard to incorporate all of the constraints related to the other vehicles in a good enough way in order for these algorithms to work fast
- finally, there are sampling based methods, which are what we will focus on today
  - these algorithms are very popular because they require a somewhat easier to compute definition of the free space
  - sampling based methods use a collision detection module that probes the free space to see if a configuration is in collision or not
  - unlike combinatorial or optimal control methods, which analyzes the whole environment, not all parts of the free space need to be explored in order to find a solution
  - explored parts are stored in a graph structure that can be searched with a graph search algorithm like Dijkstra or A*
  - two main classes of methods can be identified as sampling based:
    - discrete methods, which rely on a finite set of configurations and/or inputs, like a grid superposed on top of our configuration space
    - probabilistic methods, which rely on the probabilistic sample of a continuous configuration space
      - the set of possible configurations or states that will be explored is potentially infinite, which gives some of these methods the nice property that they are probabilistically complete and sometimes probabilistically optimal, meaning that they will always find a solution if you allow them enough computation time


- we have barely scratched the surface on all the different sorts of planning algorithms that exist
- I strongly encourage you to look up some of these algorithms and learn more about how they work

# A* Reminder

- **Q:** Check all true statements about A*.
- **A:** 
  - A* is a discrete method.
  - it uses an optimistic heuristic function to guide grid cell expansion
    - in order to be a valid, A* heuristic have to be optimistic, meaning that they should underestimate the actual cost to go from the cell to the goal
  - it always finds a solution if one exists (completeness)
  - solutions it finds are always optimal assuming an admissible heuristic
    - the more granular the discretization, the better the solution
    - we call this resolution optimal

# Hybrid A* Introduction

- one of the key differences to the planning as we talked about before is that now the word is continuous
-  a car can't really follow discrete choices on path detected with A*
  - there's a number of very sharp turns that are just irreconcilable with the motion of a car
- the fundamental problem is A* is discrete whereas the robotic world is continuous


- so the question arises, is there version of A* that can deal with the continuous nature and give us probably executable paths?
- the key to solving this with A* has to do with a state transition function

<img src="resources/hybrid_a_star_intro.png"/>

- suppose we are in a cell like this and you play a sequence of very small steps simulations using our continuous math from before, then a state over here might find itself right here in the corner of the next discrete state
- instead of assigning this just to the grid cell, an algorithm called Hybrid A* memorizes the exact $x', y', \theta'$ prime and associate it with the grid cell over here (red) the first time the grid cell is expanded
- then when expanding from the cell, it uses a specific starting point over here to figure out what the next cell might be
- now, it might happen that the same cell we used to get in A* maybe from over here (upper blue arrow) going into a different continuous polymerization of $x', y', \theta'$ but because in A* we tend to expand cells along the shortest path before we look the longer paths, we not just cut this off and never consider to stay over here again
- this leads to a lack of completeness which means there might be solutions to the navigation problem that this algorithm doesn't capture
- but it does give us correctness


- as long as our motion equations are correct, the resulting paths can be executed
- now, here's a caveat--this is an approximation and it's only correct to the point that these motions equations are correct or not correct
- but nevertheless, all paths that come out are nice, smooth, and curved paths
- every time we expand the grid cell, we memorize explicitly the continuous values of $x', y', \theta'$ with this grid cell

# Hybrid A* Tradeoffs

- **Q:** Check all true statements about Hybrid A*.
- **A:**
  - unlike regular A\*, Hybrid A\* is a continuous method - it uses a continuous search space
    - the configuration that will end up being part of the trajectory are not part of a predefined discrete set
  - it uses an optimistic heuristic function to guide grid cell expansion
  - one sacrifice made with Hybrid A* is that it does not always find a solution when one exists
    - the completeness is sacrificed because we only allow one continuous configuration per discrete cell
  - the great improvement compared with A\* is that the solution it does find are guaranteed to be driveable, though not necessarily optimal
  - compare these to the properties of A\*, with Hybrid A\* we have lost completeness and optimality guarantees in favor of drivability
    - in practice, however, this algorithm is very efficient at finding good path almost all the time

# Hybrid A* in Practice

- in the first video about Hybrid A*, the update equations that we used were somewhat generic to most X-Y field configuration spaces
  - $x(t + \Delta t) = x(t) + v\Delta t\cos(\theta)$
  - $y(t + \Delta t) = y(t) + v\Delta t\sin(\theta)$
  - $\theta(t + \Delta t) = \theta(t) + \omega \Delta t$
- in particular, we don't specify that $\omega$ is the heading rate of change
  - for some robots, it might be that we can specify any omega independently of its state but that would mean that the robot can turn around its Z axis without constraints
  - for car though, this is not very realistic, and we should probably use the bicycle model
    - this gives us an equation for $\omega$ like this: $\omega = \dfrac{v}{L} \tan(\delta(t))$
      - $L$ is the distance between the front and rear axle
      - $\delta$ is the steering angle
      - $v$ is the constant positive velocity


- for our Hybrid A* algorithm, we need to decide which configuration can be reached from the current node considered, in order to add them to the open set
- in practice, we would probably use some number of steering angles between maximum left steer and maximum right steer angle
- but the more different deltas you add the longer it takes for your algorithm to complete
- therefore, in this example, we will only choose three angles to pick delta from: the maximum left steering angle, zero steering angle and the maximum right steering angle
  - in the case of a car that can turn its wheels 35 degrees at most, that would be -35, 0 and 35
  - this allows us to expand the search tree with three new motion primitives: go straight, steer left and steer right


- let's see how that works in a robot maze
- let's say we have a maze like this

<img src="resources/hybrid_a_star_robot_maze_1.png"/>

- the red square is the goal and this is the start position (circle with arrow)
- the first thing we would do is add this cell to open list--we'll use gray to indicate that
  - then we would generate three trajectories, the one for going straight, the one for turning right, the one for turning left
  - in this case, we can discard the trajectories that go off the map or collide
  - this gives us one new cell to open, which we do, and then we mark the first cell closed which am indicating with this check-mark
- now, there is only one open cell so we choose to expend that one next
  - again, we generate the three trajectories from this stage, one of them is infeasible, so let's not keep looking at it
  - this leaves us with two new cells to open
  - we close the previous
- now there are two choices of what cell to expand
  - assuming we have some heuristic that values being closer to the goal, we choose the cell on the right, and then generate trajectories from this new state
- note how in all these steps, we're keeping track of, not just what cell we're in, but also where in that cell we are, including orientation
- in practice, we could also use a three state space that include heading
  - if we don't do that, then we're likely to prematurely close cells
- we're not doing it that way here, mostly because it's harder to visualize
- let's keep going--we have three new cells to open
  - we have to close the previous cell, find the best option in the set of open cells and repeat
  - again leaving us with three new cells to add to the open list
  - which means we can close the the previous cells and things get a little more interesting here
- our heuristic will probably suggest that we expand the lower right cell and generate trajectories from there
  - but this time, all of them would collide with the wall--therefore, we don't need to add any new cell to the open set
  - since we did expand this cell I have to mark it closed
  - then we can look at remaining open cells
  - same as before, we take a closest cell to the goal, generate trajectories, that is, that all of these collides and close that cell
  - of the remaining open cells, this upper right one is closest or the one with the best heuristic
- we repeat the process, on and on, until we reach the goal


- what we're left with is a continuous path, that might not be the smoothest solution to our planning problem
- indeed we can easily find examples of environment, where the resulting path doesn't look very good compared to what human drivers would have done in this situation
- that's not the only problem; sometimes, everyday stuff fails to even find the solution when one actually exists

<img src="resources/hybrid_a_star_robot_maze_2.png"/>

- for example, let's go back to the stage in the algorithm and consider what would happen if the map were slightly changed
- in this case, we would have made all the same decisions up until this point
- but now we have close this crucial cell
- clearly, there is a path to the goal that could be reached but in these circumstances, Hybrid A* will not find any
- we can reduce the likelihood of this problem by increasing the resolution of our grid or adding a third dimension to the search space for the heading
  - this would allow us not to close this position in the map for all headings but only for a specific range which may allow us to find the solution in this case

# Hybrid A* Heuristics

- the paper [Junior: The Stanford Entry in the Urban Challenge](https://d17h27t6h515a5.cloudfront.net/topher/2017/July/595fe838_junior-the-stanford-entry-in-the-urban-challenge/junior-the-stanford-entry-in-the-urban-challenge.pdf) is a good read overall, but Section 6.3 - Free Form Navigation is especially good and goes into detail about how the Stanford team thought about heuristics for their Hybrid A* algorithm (which they tended to use in parking lots)
- read section 6.3 of the paper and then answer the following question


- **Q:** If you have two individually admissible heuristic functions $h_1$ and $h_2$, which of the following combinations are also valid? Check the box next to all admissible combinations.
- **A:** $\dfrac{h_1 + h_2}{2}$, $\min(h_1, h_2)$, $\max(h_1, h_2)$ are valid. $h_1 + h_2$ is not valid.

# Hybrid A* Pseudocode

- the pseudocode below outlines an implementation of the A* search algorithm using the bicycle model
- the following variables and objects are used in the code but not defined there:
  - `State(x, y, theta, g, f)`: an object which stores `x`, `y` coordinates, direction `theta`, and current `g` and `f` values
  - `grid`: a 2D array of 0s and 1s indicating the area to be searched
    - 1s correspond to obstacles, and 0s correspond to free space
  - `SPEED`: the speed of the vehicle used in the bicycle model
  - `LENGTH`: the length of the vehicle used in the bicycle model
  - `NUM_THETA_CELLS`: the number of cells a circle is divided into
    - this is used in keeping track of which States we have visited already


- the bulk of the hybrid A* algorithm is contained within the `search` function
- the `expand` function takes a state and goal as inputs and returns a list of possible next states for a range of steering angles
  - this function contains the implementation of the bicycle model and the call to the A* heuristic function

```python
def expand(state, goal):
    next_states = []
    for delta in range(-35, 40, 5): 
        # Create a trajectory with delta as the steering angle using 
        # the bicycle model:

        # ---Begin bicycle model---
        delta_rad = deg_to_rad(delta)
        omega = SPEED/LENGTH * tan(delta_rad)
        next_x = state.x + SPEED * cos(theta)
        next_y = state.y + SPEED * sin(theta)
        next_theta = normalize(state.theta + omega)
        # ---End bicycle model-----

        next_g = state.g + 1
        next_f = next_g + heuristic(next_x, next_y, goal)

        # Create a new State object with all of the "next" values.
        state = State(next_x, next_y, next_theta, next_g, next_f)
        next_states.append(state)

    return next_states

def search(grid, start, goal):
    # The opened array keeps track of the stack of States objects we are 
    # searching through.
    opened = []
    # 3D array of zeros with dimensions:
    # (NUM_THETA_CELLS, grid x size, grid y size).
    closed = [[[0 for x in range(grid[0])] for y in range(len(grid))] 
        for cell in range(NUM_THETA_CELLS)]
    # 3D array with same dimensions. Will be filled with State() objects 
    # to keep track of the path through the grid. 
    came_from = [[[0 for x in range(grid[0])] for y in range(len(grid))] 
        for cell in range(NUM_THETA_CELLS)]

    # Create new state object to start the search with.
    x = start.x
    y = start.y
    theta = start.theta
    g = 0
    f = heuristic(start.x, start.y, goal)
    state = State(x, y, theta, 0, f)
    opened.append(state)

    # The range from 0 to 2pi has been discretized into NUM_THETA_CELLS cells. 
    # Here, theta_to_stack_number returns the cell that theta belongs to. 
    # Smaller thetas (close to 0 when normalized  into the range from 0 to 
    # 2pi) have lower stack numbers, and larger thetas (close to 2pi when 
    # normalized) have larger stack numbers.
    stack_num = theta_to_stack_number(state.theta)
    closed[stack_num][index(state.x)][index(state.y)] = 1

    # Store our starting state. For other states, we will store the previous 
    # state in the path, but the starting state has no previous.
    came_from[stack_num][index(state.x)][index(state.y)] = state

    # While there are still states to explore:
    while opened:
        # Sort the states by f-value and start search using the state with the 
        # lowest f-value. This is crucial to the A* algorithm; the f-value 
        # improves search efficiency by indicating where to look first.
        opened.sort(key=lambda state:state.f)
        current = opened.pop(0)

        # Check if the x and y coordinates are in the same grid cell 
        # as the goal. (Note: The idx function returns the grid index for 
        # a given coordinate.)
        if (idx(current.x) == goal[0]) and (idx(current.y) == goal.y):
            # If so, the trajectory has reached the goal.
            return path

        # Otherwise, expand the current state to get a list of possible 
        # next states.
        next_states = expand(current, goal)
        for next_s in next_states:
            # If we have expanded outside the grid, skip this next_s.
            if next_s is not in the grid:
                continue
            # Otherwise, check that we haven't already visited this cell and
            # that there is not an obstacle in the grid there.
            stack_num = theta_to_stack_number(next_s.theta)
            if closed[stack_num][idx(next_s.x)][idx(next_s.y)] == 0 
                and grid[idx(next_s.x)][idx(next_s.y)] == 0:
                # The state can be added to the opened stack.
                opened.append(next_s)
                # The stack_number, idx(next_s.x), idx(next_s.y) tuple 
                # has now been visited, so it can be closed.
                closed[stack_num][idx(next_s.x)][idx(next_s.y)] = 1
                # The next_s came from the current state, and is recorded.
                came_from[stack_num][idx(next_s.x)][idx(next_s.y)] = current
```

# Implement Hybrid A* in C++

- in this exercise, you will be provided a working implementation of a breadth first search algorithm which does not use any heuristics to improve its efficiency
- your goal is to try to make the appropriate modifications to the algorithm so that it takes advantage of heuristic functions (possibly the ones mentioned in the previous paper) to reduce the number of grid cell expansions required


- instructions:
  - modify the code in `hybrid_breadth_first.cpp`
  - note the number of expansions required to solve an empty 15x15 grid (it should be about 18,000!)
    - modify the code to try to reduce that number

- here is one possible implementation for Hybrid A* using the "distance to goal" heuristic function
- in this implementation, we have added an `f` value to the `maze_s` struct, which is set in the expand function
- additionally, we've added two new functions: `heuristic` and `compare_maze_s`
- the `compare_maze_s` function is used for comparison of `maze_s` objects when sorting the `opened` stack


- to get an even lower number of expansions, try reducing `NUM_THETA_CELLS` in `hybrid_breadth_first.h` to reduce the total number of cells that can be searched in the `closed` array
- be careful though--making `NUM_THETA_CELLS` too small might result in the algorithm being unable to find a path through the maze


- another possibility for improvement is to use the regular A* algorithm to assign a cost value to each grid cell
- this grid of costs can then be used as the heuristic function, which will lead to an extremely efficient search


- code is available in `code/01_implement_hybrid_a_star/`

# Environment Classification

- you've just implemented Hybrid A* which is one of the best algorithms for finding trajectories in unstructured environments
- an example of such an environment is a parking lot or a maze
- these environments tend to have less specific rules than highway or streets and also, lower driving speeds
- they also do not have an obvious reference path that corresponds to what we should be driving 90% of the time, because they change so much


- on the other hand, highway or streets are very structured environments and in these environments, all motion is more constrained by predefined rules, regarding how we can move on the road
- for example, the direction of traffic or lane boundaries or also speed limits
- all these rules impose constraints which have to be satisfied but also for guidance regarding what the trajectory should look like
- while A* is great at finding solutions everywhere, it does not take advantage of this information
- for example, the road structure itself can be used as a reference path

# Frenet Reminder

- before we move on, I want you to realize that although we can simply take the center of a lane as a reference line, we could also imagine using an actual nice and feasible trajectory computed with an offline planning algorithm like A* or Hybrid A*
- that trajectory would have been later smoothened
- the benefit of this is that it would allow us to have a nominal path that we know the vehicle should follow if possible, like when the corresponding portion of the lane in front of us is empty

# The Need for Time

- I'm going to emphasize why it is important to not just compute a sequence of configurations, but also to decide when we're going to be in each configuration

<img src="resources/need_for_time_path_1.png"/>

- let's say I told you that some car recently drove along this road following this trajectory
- if I asked you to describe what the driver did, you might be tempted to say that they passed the slow moving vehicle or maybe, you think they swerved to avoid a pothole
- well, let me tell you what actually happened--up here we had a broken down vehicle stopped on the road, and down here we have a quickly moving vehicle that will travel the trajectory we just saw
- I'm going to track the location of this vehicle with little yellow dots
  - add one here for T equals zero
  - in the next time step the vehicle has moved forward quite a bit and again and again
  - but after third dot not as much because it started to slow down and slowing down again
  - at this point, this vehicle would like to change lane in order to pass the stopped vehicle
  - however, at the same moment it senses a vehicle arriving behind it in the other lane at high speed
  - therefore, it has to wait until this new car has passed it in order to change lane which only gives it the option to come to a full stop and wait (T=7, T=8)
  - at T=9 when the red vehicle has passed it, the blue vehicle decide that it's safe to keep driving--it does this by turning left and accelerating, then it continues with the lane change maneuver, until it gets into the left lane where it keeps accelerating and accelerating until it gets to highway speed
  - once it has passed the stop vehicle, it gets back into the right lane and continues on its way

<img src="resources/need_for_time_path_2.png"/>

- I told you this story using discrete time increments, but the vehicle was following a continuous path that looked something like this
- if we remove the dots, we're left with the same path I asked you about in the beginning
- the problem was not incorporating timing into our planning, but is that with $s$ and $d$ and no $t$ you don't tell the full story
- you describe the motion of your vehicle in the next few seconds, but you don't actually specify where the vehicle should be at a given time within this time frame
- when we were solving static mazes' problems this was fine but highways aren't mazes
- there is traffic on highways which means that the traversal space is constantly changing over time


- to avoid collision in dynamic environment, we need to think about time as well
- that means that driving in traffic is fundamentally a three dimensional problem
- it is important to note that you have the same situation if you were using Cartesian coordinates
- for any coordinates help us simplify the problem, but do not change the nature of the problem, which is planning a trajectory in dynamic environment

# s, d, and t

- up until now you learned about generating a sequence of configurations for vehicle using algorithms like A* and Hybrid A*
- we refer to this as path planning--but what about time?
- in path planning, we imagine taking our driving surface and laying some sort of discretized grid on top of it
- then, we make plans by specifying a start state and a goal state
- we generate trajectories with algorithms like A* (discrete trajectories) or Hybrid A* (continuous trajectories)
- but the path they generate don't really take predictions into account


- for example, let's project back to when the blue vehicle first wanted to change lane
- here we will include the time as a third dimension
- one way to visualize it is to imagine looking at the road from some angle and time would be the dimension going up
- one way for a vehicle to move along the path that goes around the start vehicle, would be something like blue line
- the fast moving vehicle we perceived behind us is predicted to move in on the lane at a fast speed which would look something like red line
- that's how we know that we cannot change lane immediately
- if we try to go before the vehicle that passed us, which is another red line in the $t$ dimension, we will collide with the vehicle
- we have to wait for this vehicle to drive past us

<img src="resources/trajectories_collision_t_dimension.png"/>

- once the Red car has passed us, we can proceed with our lane change
- having a path is nice, but it is not sufficient to solve the planning problem when there are dynamic objects around us which in autonomous driving is all the time
- therefore, we have to plan not only a sequence of configuration, $s$, $d$ and $\theta$ but also have these configurations all set up in time $t$


- the trajectory $\sigma$ is a function which for any time $t$ associates a configuration of a vehicle
  - $\sigma : [0,t_f] \rightarrow \mathbb{R}$
  - $t \rightarrow [s, d, \theta]$
- now, we need a way to generate such a trajectory
- the way we're going to do this, is by separating the planning we do in the $s$ dimension, from the planning we do in a $d$ dimension
- instead of thinking about one picture in three dimension, we usually decide to reason about two pictures, each of which is in two dimensions

<img src="resources/s_d_trajectories_example.png"/>

- we can look at both trajectories simultaneously and the great thing is that we can finally visualize the whole motion of the car over time
- it's not just a path now, it's a trajectory and we can fully describe what the vehicle did within these 15 seconds

# Trajectory Matching

- **Q** Below you will see $s$ vs. $t$ and $d$ vs. $t$ graphs for four different trajectories (labeled Option A, B, C, and D). Match each set of graphs to the corresponding verbal description of the trajectory.

<img src="resources/s_d_trajectories_example_a.png"/>

<img src="resources/s_d_trajectories_example_b.png"/>

<img src="resources/s_d_trajectories_example_c.png"/>

<img src="resources/s_d_trajectories_example_d.png"/>

- **A:**
  - slow down in lane -- A
  - swerve quickly -- C
  - pass vehicle -- B
  - pull over and stop -- D

# Structured Trajectory Generation Overview

- in the next few segments, I'm going to present a method for trajectory planning that is useful in structured environments like highway driving
- to do that, I'll introduce the idea of Jerk minimization to generate a nice trajectory from a start configuration to a goal configuration
- I will show you how to generate Jerk minimizing trajectories using polynomials and then present a brief derivation on how to compute the coefficients of the polynomial that solves our problem
- then I will show you an example of what one such trajectory looks like on the highway and how we can evaluate its driveability and compare it to other trajectories
- finally, I will show you how by generating many similar jerk minimizing trajectories we can compare them and select the best one for the situation we want to drive through

# Trajectories with Boundary Conditions

- our goal is to generate continuous trajectories, but how do you do that
- let's start with a naive approach and then get more realistic
- let's say our car has been driving along the road at a constant speed for some time
  - the $s$ versus $t$ graph is just a straight line, since the velocity is constant
  - the $d$ versus $t$ time graph is flat, since the car is just staying in the center of the lane
- also note that I've shifted my axis a bit so that the origin of the frame is our current positioning in $s$, $d$ and $t$--this will end up being computationally efficient layer
- now let's say we're getting off this highway soon so we want to change lane in order to end up there (top) after some $\Delta t$ which let's say it's $10$ second
- the stop position and go position are fixed
  - they define what we call the boundary conditions of our trajectory at $t=0$ and $t=10$
  - if those were our only boundary conditions, then we might consider $s$ and $t$ trajectories looking like this

<img src="resources/trajectories_with_boundary_example.png"/>

- but it turns out that these aren't physically possible
- this kick in the slope would translate in an instantaneous jumping speed which would require infinite acceleration
- if we were to send that to our motion control module, maybe they can track this trajectory, but probably not
- and even if they tried to track it, it would result in a very strong acceleration of our car, which is both uncomfortable and dangerous
- this is why we need both continuity and smoothness in our trajectories


- but how much continuity and smoothness?
- we know we want continuity in position, since our car cannot teleport and it would not make sense to compute a trajectory starting 10 metres ahead of us
- we also know that our car cannot instantaneously change speed, therefore the speed should also be continuous
- following this logic you could also say that the acceleration should be continuous and maybe the acceleration's derivity as well and so on
- position becomes velocity, velocity because acceleration, acceleration becomes jerk, and going even further you get snap, crackle, and pop
  - it turns out that one of these quantities is directly related to human perception of comfort


- in a vehicle, humans feel discomfort when which of these values is high?
  - velocity?--no--when they cruise down the highway at high velocity, people usually don't find it less comfortable than when they drive at lower speed
    - if you think about airplanes, they move very fast and they aren't really uncomfortable
  - acceleration maybe?--that's slightly more tricky--we are indeed sensitive to acceleration but mostly when it becomes very high
    - for example, we are all used to supporting acceleration up to 1G which is the most you can usually sustain in a normal car
    - therefore, we shouldn't reach acceleration levels that are high enough to become uncomfortable
  - jerk, then?--it turns out, this is the quantity that humans perceive as uncomfortable
    - we can tolerate high acceleration but we don't like it when our acceleration changes too quickly which is high jerk
- so when we design trajectories for self-driving car, we would like it, if they were jerk minimizing

# Jerk Minimizing Trajectories

- what is a jerk minimizing trajectory?
- an interesting and useful fact to us is that it is relatively easy to compute a jerk optimal trajectory in one dimension

<img src="resources/jerk_first_order_polynomial.png"/>

- in fact, consider a function $s(t)$ defined from time $0$ to time $t_f$ and recall that jerk is a third derivative of position
  - so the total jerk accumulated over the duration of this trajectory is given by this equation (image above)
- we want to analyze both positive and negative jerk--therefore, we're going to look at total square jerk
- our problem is then to find a function $s(t)$ that minimizes this integral
- if we go through the math required to find the minimum, you'll find that all the time derivatives of $s$ of order six and more have to be zero in order for $s$ to be jerk minimal
- we can also use a helpful fact about functions, which is that any function can be written like this or more compactly like this (right side on image above)
- when you take this form of $s$ and add in the information that we have about it's time derivative, you find that all of the coefficient bigger than five or zero, which means that all minimum jerk trajectories can be represented as a fifth order polynomial
- this equation has six coefficients which means six tuneable parameters that we can choose from in order to define the shape of a 1D trajectory
  - in practice, we use them to specify the boundary conditions of our trajectory


- the things we'd like to constrain are the initial position $s_i$, velocity $\dot{s_i}$ and acceleration $\ddot{s_i}$ as well as the final position $s_f$, velocity $\dot{s_f}$ and acceleration $\ddot{s_f}$
  - this is just the 1D representing the longitude of displacement
- the same for the lateral displacement applies ($d$ instead of $s$)
- this gives us 12 variables to pick in order to fully define the motion of our vehicle in $s$ and $d$ over time
  - the initial state of our vehicle give us the longitudinal and lateral boundary conditions at $t=0$ and we get to pick the end boundary condition
- it is important to realize that although the trajectory that we generate using this method or jerk optimal for a given set of boundary conditions, they still depend heavily on these conditions
  - therefore, it would be important in the future to wisely pick these end conditions

# Derivation Overview

- in the previous video, I showed you that the position of a jerk minimizing trajectory is given by a quintic polynomial
- in this video, I'm going to present a quick and non-rigorous walk through of how we find the six coefficients that describe these polynomial
- first, we differentiate the equations of the position in order to get the equations for the velocity, and then we differentiate it again in order to get the equation for the acceleration
- now, we could just plug in our six boundary conditions to get the six equations but first we're going to do something that will make our life a little bit easier

<img src="resources/equations_derivation.png"/>

- we're going to set the initial time as zero and when we do that, we find that three or four unknowns don't need to be identified anymore ($a_0, a_1, 2a_2$), which reduces the program from six unknowns to three
- it simplifies those three trajectories by removing three of the unknowns
- we can gather the known terms into functions of the start boundary conditions to make it look like a bit cleaner
- the C1, C2, and C3 terms are functions of the initial conditions
- what remains is that we only have to identify three parameters that depend only on our $N$ boundary condition
- we know the final position, velocity and acceleration at time $t_f$ since those are all the quantities that we need
- since we know them, a more useful way to think of these equations is like this, where I have deliberately separated the known quantities from the unknown variables that we wish to determine

<img src="resources/equations_derivation_matrix.png"/>

- this is starting to look like a very solvable system of three equations
- the best way to solve this is with a matrix that looks like this, and this problem can be solved by inverting this matrix using any matrix mathematical library, which we will do later


- we have:
  - stated that Jerk Minimizing Polynomials are degree 5 polynomials with 6 tunable parameters
  - given an overview of how to calculate the coefficients for such a polynomial given the boundary conditions $[s_i, \dot{s}_i, \ddot{s}_i]$ and $[s_f, \dot{s}_f, \ddot{s}_f]$ (or the corresponding $d$ coordinates)

# Polynomial Trajectory Generation

- I want to show you how a polynomial solver to generate Jerk minimizing trajectories is being used
- let's consider the $s$ coordinates first
  - as an input, it takes the current vehicle state, the goal state, and the duration
  - as an output, it will generate six coefficients which uniquely define the polynomial that describes the longitudinal trajectory
- similarly, we feed input parameters to our solver regarding lateral configurations and compute the lateral trajectory
- we can use this approach in a situation like this one
  - you have traffic and a self-driving car
  - let's say that the requested behavior is to pass the vehicle in front of us
  - with polynomial trajectory generation, we take the current configuration of the car along with its velocity and acceleration as our start state
  - then we specify a valid goal state that leads our vehicle in the other lane
  - we feed these states into our polynomial solver along with the desired duration that will allow for the maneuver and we get a Jerk minimizing trajectory to the goal

# Implement Quintic Polynomial Solver C++

- in this exercise you will implement a quintic polynomial solver
- this will let you take boundary conditions as input and generate a polynomial trajectory which matches those conditions with minimal jerk


- your solver will take three inputs:
  - `start` - $[s_i, \dot{s_i}, \ddot{s_i}]$
  - `end` - $[s_f, \dot{s_f}, \ddot{s_f}]$
  - `T` - the duration of maneuver in seconds


- instructions
  - implement the `JMT(start, end, T)` function in `main.cpp`
- tips
  - remember, you are solving a system of equations: matrices will be helpful!--you can use The Eigen library
  - the equations for position, velocity, and acceleration are given by:
    - $s(t) = s_i + \dot{s_i}t + \frac{\ddot{s_i}}{2}t^2 + \alpha_3t^3 + \alpha_4t^4 + \alpha_5t^5$
    - $\dot{s}(t) = \dot{s_i} + \ddot{s_i}t + 3 \alpha_3t^2 + 4\alpha_4t^3 + 5\alpha_5t^4$
    - $\ddot{s}(t) = \ddot{s_i} + 6 \alpha_3t + 12\alpha_4t^2 + 20\alpha_5t^3$
  - if you evaluate these at $t=0$ you find the first three coeffecients of your JMT are:
    - $[\alpha_0, \alpha_1, \alpha_2] = [s_i, \dot{s_i}, \frac{1}{2}\ddot{s_i}]$
  - you can get the last three coefficients by evaluating these equations at $t = T$
  - when you carry out the math and write the problem in matrix form you get the following:
    - <img src="resources/quintic_polynomial_solver_matrix_quiz.gif"/>
    - all these quantities are known except for $\alpha_3, \alpha_4, \alpha_5$

- code is available in `code/02_implement_quintic_polynomial_solver/`

# What should be checked?

- **Q:** When evaluating the feasibility of a potential trajectory, which of the following quantities should be checked?
- **A:** All of these quantities should be evaluated for feasibility:
  - maximum velocity (with respect to car's capabilities and speed limit)--a car can only go so fast and speeds above the vehicle maximum are not feasible
  - minimum velocity--minimum velocity corresponds to going backwards, and that matters, too; on a lot of roads, it's illegal to go backwards
  - maximum acceleration--for maximum acceleration, we need to consider both lateral and longitudinal acceleration
    - maximum lateral accelerations need to be checked to avoid rollover or skidding
    - maximum longitudinal acceleration depends on your vehicle's powertrain
  - minimum acceleration--minimum accelerations would be negative and correspond to the vehicle maximum braking force
  - steering angle--your car can only turn so much

# Implementing Feasibility

- here we're not going to discuss how to solve this feasibility check exactly but I want to give you a few hints about how to do some initial validation for your trajectory
- in order to do so, we're going to neglect the curvature of the road and assume it is locally straight
- regarding longitudinal acceleration, we make the additional assumption that our heading is pretty much aligned with the road
  - this allows us to say that $\ddot{s}$ is the longitude acceleration of the car
  - therefore, we need to check that at any point of the trajectory, this acceleration is less than the maximum acceleration that the engine would need to supply, and more than the maximum braking deceleration of the car $a_{\text{max_breaking}} < \ddot{s} < a_{\text{max_accel}}$
  - right now, this could be a fixed value but in real life however, this should probably be computed using information about the friction of the road
- similarly, for lateral acceleration, we can check that all $\ddot{d}$ values are less than a fixed lateral acceleration value that can be set for comfort, or to avoid any risk of rollover in our car $|\ddot{d}|<a_y$
- regarding steering angle, the bicycle model tells us that there's a nice relationship between the steering angle of the car and the radius of the circle of curvature, where $L$ is the distance between the wheel axis and $R$ is the circle radius: $\tan \delta=\dfrac{L}{R}$
- the maximum curvature allowed at any point of the trajectory is given by this equation: $\tan \delta=\dfrac{L}{R} \rightarrow \kappa = \dfrac{ \tan(\delta)}{L} \rightarrow \kappa_{max} = \dfrac{\tan(\delta_{max})}{L} $
- if you remember, in the reading assignment about Hybrid A*, the curvature for path is defined like $\kappa_i = \dfrac{\Delta \phi_i}{\Delta x_i}$, where $\Delta \phi_i$ is the heading difference between two points of the trajectory and $\Delta x_i$ is the distance between them
- finally the velocity is checked against values specified by the map or the behavioral layer
  - for example, we could use the speed limit of the road in most cases--that gives us a maximal velocity but also sometimes, we need a minimal velocity like on highways where we don't want to drive too slow or we're backing up, negative $\dot{s}$ is not allowed

# Putting it All Together

- in most cases, the behavior layer might not send an exact end configuration to reach but rather an approximate one
- therefore, we want to identify a varied goal state that leads our vehicle in the approximate end configuration that's being sent to us
  - that's where our sampling based approach comes in handy
- we don't know for sure what a good end state is
- even if we receive an exact $s$ and $d$ coordinate from the behavior layer, we still have to find a good end velocity and acceleration for our car
- most times, $s$ and $d$ aren't fixed either
- therefore, we want to sample a large number of end configurations that are on the approximate desired position that we wish to drive our car to
- and we generate the corresponding Jerk minimizing trajectories for each goal configuration
- then, we discard all non-drivable trajectories, all trajectories that collide with the road boundaries or with other vehicles or pedestrians as predicted by the prediction layer


- now, we have a nice set of derivable trajectories which are all collision free and Jerk optimal in each dimension with respect to the start and goal configurations
- we need to decide which one the car should follow
- that means we need to rank them, which we are going to do by defining a cost function--what makes sense as a cost function?
  - first, consider Jerk
    - for each pair of end configurations, a quintic polynomial is Jerk optimal--but given that all the trajectories obtained are different final configuration, maybe some are better than others
    - in addition to that, we want to consider longitudinal versus lateral Jerk--what is worse for comfort?
      - turns out that side to side Jerk is more uncomfortable so we want to prioritize minimizing that over minimizing longitudinal Jerk
  - next, consider distance to obstacles
    - we would prefer being further away from the other vehicle to a situation where we are right next to their rear bumper
    - remember that we have to consider the full car dimension when looking at distance to obstacles
  - also, we want to consider distance to center line
    - we might prefer to be close to the center of the line
  - finally, we need to consider time to goal
    - we would prefer to arrive at our destination earlier rather than later
- there are plenty of other cost functions one might think of
  - the tricky part is balancing all of these costs correctly
- often, our goals conflict so the trajectory that minimizes lateral Jerk may not be the one that keeps us closest to the center line
- in practice, most of the hard work is in the details of balancing these cost functions

# Polynomial Trajectory Reading

- if you are interested in learning more about PTG, I've included a link to a paper titled [Optimal Trajectory Generation for Dynamic Street Scenarios in a Frenet Frame](https://www.researchgate.net/publication/224156269_Optimal_Trajectory_Generation_for_Dynamic_Street_Scenarios_in_a_Frenet_Frame)
- it is short and discusses some interesting (and potentially useful) topics like:
  - cost functions
  - differences between high speed and low speed trajectory generation
  - implementation of specific maneuvers relevant to highway driving like following, merging, and velocity keeping
  - how to combining lateral and longitudinal trajectories
  - aderivation of the transformation from Frenet coordinates to global coordinates (in the appendix)

# Polynomial Trajectory Generation Playground

- before you begin the final project we'd like to give you a chance to play around with cost function design and weight tweaking
- in the Python code you will be provided, you will have a working Polynomial Trajectory Generator


- getting started
  - from the project's directory, run `python evaluate_ptg.py`
  - you should see a plot similar to the one below
  - this plot shows the $s$ (x-axis) and $d$ (y-axis) trajectories followed by a vehicle in traffic (red) and a self driving car (blue)
  - in this situation, the self driving car was trying to get behind the target vehicle, but the cost functions it was using weren't weighted appropriately and so it didn't behave as expected

<img src="resources/polynomial_trajectory_generation_playground_example.png"/>


- fixing the problem(s)
  - there are 5 files in the provided code
  - you'll probably want to start by modifying cost function weights in `ptg.py` but may also want to add cost functions of your own


- file descriptions
  - `ptg.py` - the primary code for generating a polynomial trajectory for some constraints
    - this is also where weights are assigned to cost functions
    - adjusting these weights (and possibly adding new cost functions), can have a big effect on vehicle behavior
  - `cost_functions.py` - this file contains many cost functions which are used in `ptg.py` when selecting the best trajectory; some cost functions aren't yet implemented...
  - `evaluate_ptg.py` - this file sets a start state, goal, and traffic conditions and runs the PTG code
    - feel free to modify the goal, add traffic, etc... to test your vehicle's trajectory generation ability
  - `constants.py` - constants like speed limit, vehicle size, etc...
  - `helpers.py` - helper functions used by other files

- code is available in `code/03_polynomial_trajectory_generation_playground/`

# Conclusion

- in this lesson, you saw several approaches to the trajectory generation problem
- none of them is the best for every situation that a self-driving car will encounter
- in reality, most self-driving cars have several trajectory planners they can use depending on the situation
- a car may use hybrid A* in parking lots, polynomial trajectory generation on low traffic highways, and maybe several others for situations like intersections, high traffic, etc.

# Bonus Round: Path Planning

- nice work reaching the end of the path planning content!
- while you still have the project left to do here, we're also providing some additional resources and recent research on the topic that you can come back to if you have time later on


- all of these are completely optional reading - you could spend hours reading through the entirety of these!
- we suggest moving onto the project first so you have what you’ve learned fresh on your mind, before coming back to check these out


- we've categorized these papers to hopefully help you narrow down which ones might be of interest, as well including their Abstract section, which summarizes the paper

## Indoors

Intention-Net: Integrating Planning and Deep Learning for Goal-Directed Autonomous Navigation by S. W. Gao, et. al. https://arxiv.org/abs/1710.05627

***Abstract:*** *How can a delivery robot navigate reliably to a destination in a new office building, with minimal prior information? To tackle this challenge, this paper introduces a two-level hierarchical approach, which integrates model-free deep learning and model-based path planning. At the low level, a neural-network motion controller, called the intention-net, is trained end-to-end to provide robust local navigation. The intention-net maps images from a single monocular camera and "intentions" directly to robot controls. At the high level, a path planner uses a crude map, e.g., a 2-D floor plan, to compute a path from the robot's current location to the goal. The planned path provides intentions to the intention-net. Preliminary experiments suggest that the learned motion controller is robust against perceptual uncertainty and by integrating with a path planner, it generalizes effectively to new environments and goals.*

## City Navigation

Learning to Navigate in Cities Without a Map by P. Mirowski, et. al. https://arxiv.org/abs/1804.00168

***Abstract:*** *Navigating through unstructured environments is a basic capability of intelligent creatures, and thus is of fundamental interest in the study and development of artificial intelligence. Long-range navigation is a complex cognitive task that relies on developing an internal representation of space, grounded by recognizable landmarks and robust visual processing, that can simultaneously support continuous self-localization ("I am here") and a representation of the goal ("I am going there"). Building upon recent research that applies deep reinforcement learning to maze navigation problems, we present an end-to-end deep reinforcement learning approach that can be applied on a city scale. [...] We present an interactive navigation environment that uses Google StreetView for its photographic content and worldwide coverage, and demonstrate that our learning method allows agents to learn to navigate multiple cities and to traverse to target destinations that may be kilometers away. [...]*

## Intersections

A Look at Motion Planning for Autonomous Vehicles at an Intersection by S. Krishnan, et. al. https://arxiv.org/abs/1806.07834

***Abstract:*** *Autonomous Vehicles are currently being tested in a variety of scenarios. As we move towards Autonomous Vehicles, how should intersections look? To answer that question, we break down an intersection management into the different conundrums and scenarios involved in the trajectory planning and current approaches to solve them. Then, a brief analysis of current works in autonomous intersection is conducted. With a critical eye, we try to delve into the discrepancies of existing solutions while presenting some critical and important factors that have been addressed. Furthermore, open issues that have to be addressed are also emphasized. We also try to answer the question of how to benchmark intersection management algorithms by providing some factors that impact autonomous navigation at intersection.*

## Planning in Traffic with Deep Reinforcement Learning

DeepTraffic: Crowdsourced Hyperparameter Tuning of Deep Reinforcement Learning Systems for Multi-Agent Dense Traffic Navigation by L. Fridman, J. Terwilliger and B. Jenik https://arxiv.org/abs/1801.02805

***Abstract:*** *We present a traffic simulation named DeepTraffic where the planning systems for a subset of the vehicles are handled by a neural network as part of a model-free, off-policy reinforcement learning process. The primary goal of DeepTraffic is to make the hands-on study of deep reinforcement learning accessible to thousands of students, educators, and researchers in order to inspire and fuel the exploration and evaluation of deep Q-learning network variants and hyperparameter configurations through large-scale, open competition. This paper investigates the crowd-sourced hyperparameter tuning of the policy network that resulted from the first iteration of the DeepTraffic competition where thousands of participants actively searched through the hyperparameter space.*